In [3]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=57a279b71211a96ac102643ec0efaec0e5aa33364b181cef36c590a4cf4a8b88
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [4]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T

In [77]:
import random
import requests
import time
import json
from pandas import json_normalize

In [5]:
spark=SparkSession.builder.appName('test').getOrCreate()

In [78]:
switch=False

In [79]:
url = "https://api.carbonintensity.org.uk/intensity"

while switch:
  response = requests.get(url)
  json_normalize(response.json())
  message = response.json()
  json_normalize(message['data']).rename(columns={"intensity.forecast": "intensity_forecast",\
                                                  "intensity.actual": "intensity_actual",\
                                                  "intensity.percentile": "intensity_percentile",\
                                                  "intensity.index": "intensity_index"\
                                                  }).to_csv("intensity_"+str(random.randint(1,10000))+".csv", index=False)
  time.sleep(60)







ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt



In [ ]:
schema = T.StructType("data":)

In [22]:
#create Dataframe representing the stream of input lines from connection to localhost:9999
intensity = spark.readStream.format('json').options(path=".").load()
intensity

IllegalArgumentException: Schema must be specified when creating a streaming source DataFrame. If some files already exist in the directory, then depending on the file format you may be able to create a static DataFrame on that directory with 'spark.read.load(directory)' and infer schema from it.

In [24]:
spark.read.load(".", format="json").printSchema()

root
 |-- _corrupt_record: string (nullable = true)



5914